In [1]:
!pip install ../input/mtcnn-package/mtcnn-0.1.0-py3-none-any.whl

Processing /kaggle/input/mtcnn-package/mtcnn-0.1.0-py3-none-any.whl


In [2]:
import pandas as pd
import keras
import os
import numpy as np
from sklearn.metrics import log_loss
from keras import Model,Sequential
from keras.layers import *
from keras.optimizers import *
from sklearn.model_selection import train_test_split
import cv2
from tqdm.notebook import tqdm
import glob
from mtcnn import MTCNN

Using TensorFlow backend.


In [3]:
train0 = pd.read_json('../input/deepfake/metadata0.json')
train1 = pd.read_json('../input/deepfake/metadata1.json')
train2 = pd.read_json('../input/deepfake/metadata2.json')
train3 = pd.read_json('../input/deepfake/metadata3.json')
train4 = pd.read_json('../input/deepfake/metadata4.json')
train5 = pd.read_json('../input/deepfake/metadata5.json')
train6 = pd.read_json('../input/deepfake/metadata6.json')
train7 = pd.read_json('../input/deepfake/metadata7.json')
train8 = pd.read_json('../input/deepfake/metadata8.json')
train9 = pd.read_json('../input/deepfake/metadata9.json')
train10 = pd.read_json('../input/deepfake/metadata10.json')
train11 = pd.read_json('../input/deepfake/metadata11.json')
train12 = pd.read_json('../input/deepfake/metadata12.json')
train13 = pd.read_json('../input/deepfake/metadata13.json')
train14 = pd.read_json('../input/deepfake/metadata14.json')
train15 = pd.read_json('../input/deepfake/metadata15.json')
train16 = pd.read_json('../input/deepfake/metadata16.json')
train17 = pd.read_json('../input/deepfake/metadata17.json')
train18 = pd.read_json('../input/deepfake/metadata18.json')
train19 = pd.read_json('../input/deepfake/metadata19.json')
train20 = pd.read_json('../input/deepfake/metadata20.json')
train21 = pd.read_json('../input/deepfake/metadata21.json')
train22 = pd.read_json('../input/deepfake/metadata22.json')
train23 = pd.read_json('../input/deepfake/metadata23.json')
train24 = pd.read_json('../input/deepfake/metadata24.json')
train25 = pd.read_json('../input/deepfake/metadata25.json')
train26 = pd.read_json('../input/deepfake/metadata26.json')
train27 = pd.read_json('../input/deepfake/metadata27.json')
train28 = pd.read_json('../input/deepfake/metadata28.json')
val1 = pd.read_json('../input/deepfake/metadata29.json')
val2 = pd.read_json('../input/deepfake/metadata30.json')
LABELS = ['REAL','FAKE']
trains = [train0, train1, train2, train3, train4,
          train5, train6, train7, train8, train9, train10,
          train11, train12, train13, train14, train15,
          train16, train17, train18, train19, train20, train21,
          train22, train23, train24, train25, train26, train27,
          train28]
vals = [val1, val2]
nums = list(range(len(trains)+1))
val_nums = [29,30]

In [4]:
def get_path(num, n):
    num = str(num)
    if len(num) == 2:
        path = '../input/deepfake/DeepFake' + num + '/DeepFake' + num + '/' + x.replace('.mp4', '') + '.jpg'
    else:
        path='../input/deepfake/DeepFake0' + num + '/DeepFake0' + num + '/' + x.replace('.mp4', '') + '.jpg'
    if not os.path.exists(path):
       raise Exception
    return path
paths = []
y = []
for train,num in tqdm(zip(trains, nums), total=len(trains)):
    images = list(train.columns.values)
    for x in images:
        try:
            paths.append(get_path(num, x))
            y.append(LABELS.index(train[x]['label']))
        except Exception as err:
            pass

val_paths = []
val_y = []
for val,num in tqdm(zip(vals, val_nums), total=len(vals)):
    images = list(val.columns.values)
    for x in images:
        try:
            val_paths.append(get_path(num, x))
            val_y.append(LABELS.index(val[x]['label']))
        except Exception as err:
            pass

In [5]:
print('There are '+ str(y.count(1)) + ' fake train samples')
print('There are '+ str(y.count(0)) + ' real train samples')
print('There are '+ str(val_y.count(1)) + ' fake val samples')
print('There are '+ str(val_y.count(0)) + ' real val samples')

There are 54474 fake train samples
There are 9733 real train samples
There are 3705 fake val samples
There are 925 real val samples


In [6]:
import random
real = []
fake = []
for m, n in zip(paths, y):
    if n == 0:
        real.append(m)
    else:
        fake.append(m)
fake = random.sample(fake, len(real))
paths, y = [], []
for x in real:
    paths.append(x)
    y.append(0)
for x in fake:
    paths.append(x)
    y.append(1)

In [7]:
real = []
fake = []
for m, n in zip(val_paths, val_y):
    if n == 0:
        real.append(m)
    else:
        fake.append(m)
fake = random.sample(fake, len(real))
val_paths,val_y = [],[]
for x in real:
    val_paths.append(x)
    val_y.append(0)
for x in fake:
    val_paths.append(x)
    val_y.append(1)

In [8]:
print('There are ' + str(y.count(1)) + ' fake train samples')
print('There are ' + str(y.count(0)) + ' real train samples')
print('There are ' + str(val_y.count(1)) + ' fake val samples')
print('There are ' + str(val_y.count(0)) + ' real val samples')

There are 9733 fake train samples
There are 9733 real train samples
There are 925 fake val samples
There are 925 real val samples


In [9]:
def read_img(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
X = []
for img in tqdm(paths):
    X.append(read_img(img))
val_X = []
for img in tqdm(val_paths):
    val_X.append(read_img(img))

In [10]:
import random
def shuffle(X, y):
    new_train = []
    for m,n in zip(X, y):
        new_train.append([m,n])
    random.shuffle(new_train)
    X,y = [],[]
    for x in new_train:
        X.append(x[0])
        y.append(x[1])
    return X,y

In [11]:
X, y = shuffle(X, y)
val_X, val_y = shuffle(val_X, val_y)

In [12]:
def InceptionLayer(a, b, c, d):
    def func(x):
        x1 = Conv2D(a, (1, 1), padding = 'same', activation = 'elu')(x)
        
        x2 = Conv2D(b, (1, 1), padding = 'same', activation = 'elu')(x)
        x2 = Conv2D(b, (3, 3), padding = 'same', activation = 'elu')(x2)
            
        x3 = Conv2D(c, (1, 1), padding = 'same', activation = 'elu')(x)
        x3 = Conv2D(c, (3, 3), dilation_rate = 2, strides = 1, padding = 'same', activation = 'elu')(x3)
        
        x4 = Conv2D(d, (1, 1), padding='same', activation='elu')(x)
        x4 = Conv2D(d, (3, 3), dilation_rate = 3, strides = 1, padding = 'same', activation = 'elu')(x4)
        y = Concatenate(axis = -1)([x1, x2, x3, x4])
            
        return y
    return func
    
def define_model():
    x = Input(shape = (256, 256, 3))
    
    x1 = InceptionLayer(1, 4, 4, 2)(x)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling2D(pool_size = (2, 2), padding = 'same')(x1)
    
    x2 = InceptionLayer(2, 4, 4, 2)(x1)
    x2 = BatchNormalization()(x2)        
    x2 = MaxPooling2D(pool_size = (2, 2), padding = 'same')(x2)        
        
    x3 = Conv2D(16, (5, 5), padding = 'same', activation = 'elu')(x2)
    x3 = BatchNormalization()(x3)
    x3 = MaxPooling2D(pool_size = (2, 2), padding = 'same')(x3)
        
    x4 = Conv2D(16, (5, 5), padding = 'same', activation = 'elu')(x3)
    x4 = BatchNormalization()(x4)
    x4 = MaxPooling2D(pool_size = (4, 4), padding = 'same')(x4)
        
    y = Flatten()(x4)
    y = Dropout(0.5)(y)
    y = Dense(16)(y)
    y = LeakyReLU(alpha = 0.1)(y)
    y = Dropout(0.5)(y)
    y = Dense(1, activation = 'sigmoid')(y)
    model=Model(inputs = x, outputs = y)
    model.compile(loss = 'binary_crossentropy', optimizer = Adam(lr=1e-4))
    #model.summary()
    return model

# Train Model

In [13]:
from keras.callbacks import LearningRateScheduler
lrs = [1e-3, 5e-4, 1e-4]
def schedule(epoch):
    return lrs[epoch]

In [14]:
import keras.backend as K
import gc
kfolds = 5
models = []
losses = []
for i in range(kfolds):
    model = define_model()
    if i == 0:
        model.summary()
    model.load_weights('../input/meso-pretrain/MesoInception_DF')
    model.fit([X], [y], epochs=2, callbacks = [LearningRateScheduler(schedule)])
    models.append(model)
    pred=model.predict([val_X])
    loss=log_loss(val_y, pred)
    losses.append(loss)
    print('fold ' + str(i) + ' model loss: ' + str(loss))
    K.clear_session()
    del model
    gc.collect()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 4)  16          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 256, 256, 4)  16          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 256, 256, 2)  8           input_1[0][0]                    
____________________________________________________________________________________________

In [ ]:
import keras.backend as K
import gc
kfolds = 5
for i in range(kfolds):
    model = define_model()
    if i == 0:
        model.summary()
    model.load_weights('../input/meso-pretrain/MesoInception_F2F')
    model.fit([X], [y], epochs=2, callbacks = [LearningRateScheduler(schedule)])
    models.append(model)
    pred = model.predict([val_X])
    loss = log_loss(val_y, pred)
    losses.append(loss)
    print('fold ' + str(i) + ' model loss: ' + str(loss))
    K.clear_session()
    del model
    gc.collect()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 4)  16          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 256, 256, 4)  16          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 256, 256, 2)  8           input_1[0][0]                    
____________________________________________________________________________________________

In [ ]:
def prediction_pipline(X):
    preds = []
    for model in models:
        pred = model.predict([X])
        preds.append(pred)
    return sum(preds) / len(preds)
def larger_range(model_pred,time):
    return (((model_pred - 0.5) * time) + 0.5).clip(0.35, 0.65)

In [ ]:
model_pred = prediction_pipline(val_X)

# Validation

Some baselines:

In [ ]:
random_pred=np.random.random(len(val_X))
print('random loss: ' + str(log_loss(val_y,random_pred.clip(0.45,0.65))))
allone_pred=np.array([1 for _ in range(len(val_X))])
print('1 loss: ' + str(log_loss(val_y,allone_pred)))
allzero_pred=np.array([0 for _ in range(len(val_X))])
print('0 loss: ' + str(log_loss(val_y,allzero_pred)))
allpoint5_pred=np.array([0.5 for _ in range(len(val_X))])
print('0.5 loss: ' + str(log_loss(val_y,allpoint5_pred)))

Model loss

In [ ]:
print('Simple Averaging Loss: '+str(log_loss(val_y,model_pred.clip(0.35,0.65))))

Take a look at predictions

In [ ]:
print(larger_range(model_pred,2).mean())
print(larger_range(model_pred,2).std())

In [ ]:
def check_answers(pred, real,num):
    for i, (x, y) in enumerate(zip(pred, real)):
        correct_incorrect = 'correct' if round(float(x), 0) == round(float(y), 0) else 'incorrect'
        print(correct_incorrect + ' prediction: ' + str(x[0]) + ', answer: ' + str(y))
        if i > num:
            return
def correct_precentile(pred, real):
    correct = 0
    incorrect = 0
    for x, y in zip(pred, real):
        if round(float(x), 0) == round(float(y), 0):
            correct += 1
        else:
            incorrect += 1
    print('number correct: ' + str(correct) + ', number incorrect: ' + str(incorrect))
    print(str(round(correct / len(real) * 100, 1)) + '% correct' + ', ' + str(round(incorrect / len(real) * 100, 1)) + '% incorrect')
check_answers(larger_range(model_pred, 2), val_y, 10)
correct_precentile(larger_range(model_pred, 2), val_y)

In [ ]:
del X, y, val_X, val_y

# Make submission

In [ ]:
MAX_SKIP = 10
NUM_FRAME = 150
test_dir = '/kaggle/input/deepfake-detection-challenge/test_videos/'
filenames = os.listdir(test_dir)
prediction_filenames = filenames
test_video_files = [test_dir + x for x in filenames]
detector = MTCNN()
def detect_face(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    final = []
    detected_faces_raw = detector.detect_faces(img)
    if detected_faces_raw == []:
        return []
    confidences = []
    for n in detected_faces_raw:
        x, y, w, h = n['box']
        final.append([x, y, w, h])
        confidences.append(n['confidence'])
    if max(confidences) < 0.7:
        return []
    max_conf_coord = final[confidences.index(max(confidences))]
    return max_conf_coord
def crop(img, x, y, w, h):
    x -= 40
    y -= 40
    w += 80
    h += 80
    if x < 0:
        x = 0
    if y <= 0:
        y = 0
    return cv2.cvtColor(cv2.resize(img[y:y + h, x:x + w], (256, 256)), cv2.COLOR_BGR2RGB)
def detect_video(video):
    v_cap = cv2.VideoCapture(video)
    v_cap.set(1, NUM_FRAME)
    success, vframe = v_cap.read()
    vframe = cv2.cvtColor(vframe, cv2.COLOR_BGR2RGB)
    bounding_box = detect_face(vframe)
    if bounding_box == []:
        count = 0
        current = NUM_FRAME
        while bounding_box == [] and count < MAX_SKIP:
            current += 1
            v_cap.set(1, current)
            success, vframe = v_cap.read()
            vframe = cv2.cvtColor(vframe, cv2.COLOR_BGR2RGB)
            bounding_box=detect_face(vframe)
            count += 1
        if bounding_box == []:
            print('no faces found')
            prediction_filenames.remove(video.replace('/kaggle/input/deepfake-detection-challenge/test_videos/',''))
            return None
    x,y,w,h=bounding_box
    v_cap.release()
    return crop(vframe,x,y,w,h)
test_X = []
for video in tqdm(test_video_files):
    x = detect_video(video)
    if x is None:
        continue
    test_X.append(x)

In [ ]:
df_test = pd.read_csv('/kaggle/input/deepfake-detection-challenge/sample_submission.csv')
df_test['label'] = 0.5
preds = prediction_pipline(test_X).clip(0.35, 0.65)
for pred, name in zip(preds, prediction_filenames):
    name = name.replace('/kaggle/input/deepfake-detection-challenge/test_videos/','')
    df_test.iloc[list(df_test['filename']).index(name), 1] = pred

In [ ]:
print(preds.mean())
print(preds.std())
print(preds[:10])

In [ ]:
df_test.head()

In [ ]:
df_test.to_csv('submission.csv', index=False)

# Further Work
1. Do some more hyperparamater tuning
2. Train on the whole video(and maybe also sound)
3. Try LSTM-CNN
4. K Folds(I will try it later when I upload more data)